In [1]:
%matplotlib notebook
%pylab

Using matplotlib backend: nbAgg
Populating the interactive namespace from numpy and matplotlib


## Timing and accuracy results for DG, CG and WPA
<hr style="border-width:4px; border-color:coral"></hr>

In [2]:
import pandas 

In [3]:
idx = pandas.IndexSlice

# Walltime     : time TFinal (global time)
# Prep         : one time step up time
# Integration  : 
# time steps   : Number of time steps to time tfinal
# dt           : Time step size
# CFL          : CFL number
# Errors       : (1-norm, 2-norm and inf-norm) errors

ex_list = ['advection','SWE']

cols = ['walltime', 'Prep', 'Integ', 'time steps', 'dt', 'cfl','1-norm','2-norm','inf-norm']

order = [1,2,3,4,6]
elements = [8,16,32,64,128,256,512, 1024]
iterables = [ex_list, ['DG','CG','WPA'],['exact','inexact','w/lim','wout/lim'], order, elements]

index = pandas.MultiIndex.from_product(iterables,names=['example','method','integration','order','elements'])
df = pandas.DataFrame(index=index,columns=cols).sort_index()

df.loc[idx['SWE','CG',['exact','inexact']]]

walltime Prep Integ time steps   dt  \
example method integration order elements                                       
SWE     CG     exact       1     8             NaN  NaN   NaN        NaN  NaN   
                                 16            NaN  NaN   NaN        NaN  NaN   
                                 32            NaN  NaN   NaN        NaN  NaN   
                                 64            NaN  NaN   NaN        NaN  NaN   
                                 128           NaN  NaN   NaN        NaN  NaN   
...                                            ...  ...   ...        ...  ...   
               inexact     6     64            NaN  NaN   NaN        NaN  NaN   
                                 128           NaN  NaN   NaN        NaN  NaN   
                                 256           NaN  NaN   NaN        NaN  NaN   
                                 512           NaN  NaN   NaN        NaN  NaN   
                                 1024          NaN  NaN   NaN        NaN  NaN   

                                           cfl 1-norm 2-norm inf-norm  
example method integration order elements                              
SWE     CG     exact       1     8         NaN    NaN    NaN      NaN  
                                 16        NaN    NaN    NaN      NaN  
                                 32        NaN    NaN    NaN      NaN  
                                 64        NaN    NaN    NaN      NaN  
                                 128       NaN    NaN    NaN      NaN  
...                                        ...    ...    ...      ...  
               inexact     6     64        NaN    NaN    NaN      NaN  
                                 128       NaN    NaN    NaN      NaN  
                                 256       NaN    NaN    NaN      NaN  
                                 512       NaN    NaN    NaN      NaN  
                                 1024      NaN    NaN    NaN      NaN  

[80 rows x 9 columns]

## Read in binary WPA data
<hr style="border-width:2px; border-color:black"></hr>

Read in data created during simulation.  The data is output in binary using a Numpy dynamic type `dtype`.  The data has to then be read in using the same format.  The file structure is : 

    Row 1  : nlen               Number of N values in convergence study

The following block then repeats four times for order=(1,2), limiter=(0,1)

Block :

      dtype([('order',int),('limiter',int)])   
      
      # Followed by N rows of 
      dtype([('N',int),('wall','d'), ('setup','d'),('integ','d'),('M',int),('dt','d'),('cfl','d'),
                  ('1-norm','d'),('2-norm','d'),('inf-norm','d')])
                  


In [4]:
output_file = 'WPA/wpa_adv_data.dat'

# Open file as a binary file
fout = open(output_file,"rb")  
nlen = fromfile(fout,dtype=int,count=1)

# Create a data type for storing results;  use 'd' for double precision entries
dt_data = dtype([('N',int),('wall','d'), ('setup','d'),('integ','d'),('M',int),('dt','d'),('cfl','d'),
                  ('1-norm','d'),('2-norm','d'),('inf-norm','d')])  

# Create a nested data type
dt_info = dtype([('order',int),('limiter',int)])
dt_block = dtype([('info',dt_info),('data',(dt_data,nlen))])

# Read in four sets of data (orders 1,2 and limiters 0,1)
wpa_data = fromfile(fout,dtype=dt_block, count=4)
fout.close()

cols = ['walltime','Prep','Integ','time steps','dt', 'cfl', '1-norm','2-norm','inf-norm']

# Store data in DataFrame
for k in range(4):     # order=(1,2) x lim=(0,1)
    order = wpa_data[k]['info']['order']
    lim = wpa_data[k]['info']['limiter']
    if lim == 0:        
        limidx = 'wout/lim'
    else:
        limidx = 'w/lim'
    
    for d in wpa_data[k]['data']:
        N = d[0]
        a = [d[i] for i in range(1,len(d))]
        df.loc[idx['advection','WPA',limidx,order,N],cols] = a


# Format the data when displaying the DataFrame

fstr = {'walltime' : "{:.4e}".format, 
        'Prep' : "{:.2e}".format,
        "dt" : "{:.4e}", 
        'Integ' : "{:.4e}".format,
        'cfl' : "{:.2f}".format, 
        '1-norm' : '{:.4e}'.format,
        '2-norm' : '{:.4e}'.format,
        'inf-norm' : '{:.4e}'.format}
        
df_wpa = df.loc[idx['advection','WPA',['w/lim','wout/lim'],1:2],:]
df_wpa.style.format(fstr)

FileNotFoundError: [Errno 2] No such file or directory: 'WPA/wpa_adv_data.dat'

## Read in binary CG/DG data
<hr style="border-width:2px; border-color:black"></hr>

Read in data created during simulation.  The data is output in binary using a Numpy dynamic type `dtype`.  The data has to then be read in using the same format.  The file structure is : 

    Row 1  : nlen               Number of N values in convergence study

The following block then repeats four times for order=(1,2), limiter=(0,1)

Block :

      dtype([('order',int),('limiter',int)])   
      
      # Followed by N rows of 
      dtype([('N',int),('wall','d'), ('setup','d'),('integ','d'),('M',int),('dt','d'),('cfl','d'),
                  ('1-norm','d'),('2-norm','d'),('inf-norm','d')])
                  


In [ ]:
output_file = 'CGDG/dg_adv_data.dat'

# Open file as a binary file
fout = open(output_file,"rb")  
nlen = fromfile(fout,dtype=int,count=1)

# Create a data type for storing results;  use 'd' for double precision entries
dt_data = dtype([('N',int),('wall','d'), ('setup','d'),('integ','d'),('M',int),('dt','d'),('cfl','d'),
                  ('1-norm','d'),('2-norm','d'),('inf-norm','d')])  

# Create a nested data type
dt_info = dtype([('order',int),('integType',int)])
dt_block = dtype([('info',dt_info),('data',(dt_data,nlen))])

# Read in four sets of data (orders 1,2 and limiters 0,1)
dg_data = fromfile(fout,dtype=dt_block, count=4)
fout.close()

cols = ['walltime','Prep','Integ','time steps','dt', 'cfl', '1-norm','2-norm','inf-norm']

# Store data in DataFrame
for k in range(4):     # order=(1,2) x lim=(0,1)
    order = dg_data[k]['info']['order']
    Itype = dg_data[k]['info']['integType']
    if Itype == 1:        
        typex = 'inexact'
    else:
        typex = 'exact'
    
    for d in dg_data[k]['data']:
        N = d[0]
        a = [d[i] for i in range(1,len(d))]
        df.loc[idx['advection','DG',typex,order,N],cols] = a


# Format the data when displaying the DataFrame

fstr = {'walltime' : "{:.4e}".format, 
        'Prep' : "{:.2e}".format,
        "dt" : "{:.4e}", 
        'Integ' : "{:.4e}".format,
        'cfl' : "{:.2f}".format, 
        '1-norm' : '{:.4e}'.format,
        '2-norm' : '{:.4e}'.format,
        'inf-norm' : '{:.4e}'.format}
        
df_dg = df.loc[idx['advection','DG',['exact','inexact'],1:2],:]
df_dg.style.format(fstr)

In [ ]:
output_file = 'CGDG/cg_adv_data.dat'

# Open file as a binary file
fout = open(output_file,"rb")  
nlen = fromfile(fout,dtype=int,count=1)

# Create a data type for storing results;  use 'd' for double precision entries
dt_data = dtype([('N',int),('wall','d'), ('setup','d'),('integ','d'),('M',int),('dt','d'),('cfl','d'),
                  ('1-norm','d'),('2-norm','d'),('inf-norm','d')])  

# Create a nested data type
dt_info = dtype([('order',int),('integType',int)])
dt_block = dtype([('info',dt_info),('data',(dt_data,nlen))])

# Read in four sets of data (orders 1,2 and integType 1,2)
cg_data = fromfile(fout,dtype=dt_block, count=4)
fout.close()

cols = ['walltime','Prep','Integ','time steps','dt', 'cfl', '1-norm','2-norm','inf-norm']

# Store data in DataFrame
for k in range(4):                  # order = (1,2) x integType = (1,2)
    order = cg_data[k]['info']['order']
    Itype = cg_data[k]['info']['integType']
    if Itype == 1:        
        typex = 'inexact'
    else:
        typex = 'exact'
    
    for d in cg_data[k]['data']:
        N = d[0]
        a = [d[i] for i in range(1,len(d))]
        df.loc[idx['advection','CG',typex,order,N],cols] = a


# Format the data when displaying the DataFrame

fstr = {'walltime' : "{:.4e}".format, 
        'Prep' : "{:.2e}".format,
        "dt" : "{:.4e}", 
        'Integ' : "{:.4e}".format,
        'cfl' : "{:.2f}".format, 
        '1-norm' : '{:.4e}'.format,
        '2-norm' : '{:.4e}'.format,
        'inf-norm' : '{:.4e}'.format}
        
df_cg = df.loc[idx['advection','CG',['exact','inexact'],1:2],:]
df_cg.style.format(fstr)

## Add extra columns
<hr style="border-width:2px; border-color:black"></hr>

In [ ]:
# Add a DOF column

integration = df.index.levels[2].values
orders = df.index.levels[3].values
elements = df.index.levels[4].values

for o in orders:
    # Is this right?  
    for I in integration:
        df.loc[idx['advection','DG',I,o,:],'DOF'] = (o + 1)*array(elements)
        df.loc[idx['advection','CG',I,o,:],'DOF'] = o*array(elements) + 1
    
# One degree of freedom per element
df.loc[idx['advection','WPA','w/lim',1,:],'DOF'] = elements
df.loc[idx['advection','WPA','w/lim',2,:],'DOF'] = elements

# display data frame
df.loc[idx['advection','DG','exact',:,:],:]

## Using a MultiIndex table

<hr style="border-width:4px; border-color:coral"/>

In [ ]:
# Extract all of the data for one example
df.loc['advection']

In [ ]:
df.loc[('advection','WPA',('w/lim','wout/lim')),:]

In [ ]:
df.loc[('advection','DG',('exact')),['walltime','Prep','1-norm','cfl']]


In [ ]:
cols = ['walltime','Prep','time steps','1-norm'] 
df.loc[idx['advection',:,('exact','wout/lim')],cols].unstack(3)

In [ ]:
df.loc[idx['advection',('DG','CG'),'exact',2],:].\
       style.background_gradient(subset=['walltime'],cmap='YlOrBr')

## Plotting : CG vs. DG
<hr style="border-width:4px; border-color:coral"></hr>

In [ ]:
# set up nice tick marks for log data
def set_xticks(P):
    p0 = log2(P[0])
    p1 = log2(P[-1])
    xlim([2**(p0-1), 2**(p1+1)])
    
    Pstr = (['{:d}'.format(int(p)) for p in P])
    xticks(P,Pstr)

In [ ]:
figure(1)
clf()

order = 1
# Retrieve element values - level 3 in the Multiindex
Nv = df.index.levels[4].values

# Retrieve time values 
time_dg = df.loc[idx['advection','DG','exact',order,:],'walltime'].values
time_cg = df.loc[idx['advection','CG','exact',order,:],'walltime'].values
time_wpa = df.loc[idx['advection','WPA','w/lim',order,:],'walltime'].values

loglog(Nv,time_cg,'r.-',markersize=10,label='CG')
loglog(Nv,time_dg,'b.-',markersize=10,label='DG')
loglog(Nv,time_wpa,'g.-',markersize=10,label='WPA')

set_xticks(Nv)

title('Timing comparison between CG/DG(Exact) and WPA(w/lim)')
xlabel('# of elements')
ylabel('Walltime (s)')

legend()

figure(2)
clf()

# Retrieve element values - level 3 in the Multiindex
Nv = df.index.levels[4].values

# Retrieve time values 
time_dg = df.loc[idx['advection','DG','inexact',order,:],'walltime'].values
time_cg = df.loc[idx['advection','CG','inexact',order,:],'walltime'].values
time_wpa = df.loc[idx['advection','WPA','wout/lim',order,:],'walltime'].values

loglog(Nv,time_cg,'r.-',markersize=10,label='CG')
loglog(Nv,time_dg,'b.-',markersize=10,label='DG')
loglog(Nv,time_wpa,'g.-',markersize=10,label='WPA')

set_xticks(Nv)

title('Timing comparison between CG/DG(Inexact) and WPA(wout/lim)')
xlabel('# of elements')
ylabel('Walltime (s)')

legend()
show()

In [ ]:
figure(3)
clf()

# Retrieve element values - level 3 in the Multiindex
Nv = df.index.levels[4].values

# Retrieve time values 
e2_dge = df.loc[idx['advection','DG','exact',order,:],'2-norm'].values
e2_cge = df.loc[idx['advection','CG','exact',order,:],'2-norm'].values
e2_wpaw = df.loc[idx['advection','WPA','w/lim',order,:],'2-norm'].values

loglog(time_cg,e2_cge,'r.-',markersize=10,label='CG')
loglog(time_dg,e2_dge,'b.-',markersize=10,label='DG')
loglog(time_wpa,e2_wpaw,'g.-',markersize=10,label='WPA')

#set_xticks(Nv)

title('Timing comparison between CG/DG(Exact) and WPA(w/lim)')
xlabel('Walltime (s)')
ylabel('L2_error')


legend()

figure(4)
clf()

# Retrieve element values - level 3 in the Multiindex
Nv = df.index.levels[4].values

# Retrieve time values 
e2_dgi = df.loc[idx['advection','DG','inexact',order,:],'2-norm'].values
e2_cgi = df.loc[idx['advection','CG','inexact',order,:],'2-norm'].values
e2_wpano = df.loc[idx['advection','WPA','wout/lim',order,:],'2-norm'].values

loglog(time_cg,e2_cgi,'r.-',markersize=10,label='CG')
loglog(time_dg,e2_dgi,'b.-',markersize=10,label='DG')
loglog(time_wpa,e2_wpano,'g.-',markersize=10,label='WPA')

#set_xticks(Nv)

title('Timing comparison between CG/DG(Inexact) and WPA(wout/lim)')
xlabel('Walltime (s)')
ylabel('L2_error')

legend()
show()

In [ ]:
figure(5)
clf()   

Nv = df.index.levels[4].values

# DG exact 
p = polyfit(log(Nv), log(e2_dge.astype(float)), 1)
loglog(Nv, e2_dge, '.-',markersize=10, label = 'DG, rate = {:.2f}'.format(p[0]))
loglog(Nv, exp(polyval(p,log(Nv))), '--')

# CG exact
p = polyfit(log(Nv), log(e2_cge.astype(float)), 1)
loglog(Nv, e2_cge, '.-',markersize=10, label = 'CG, rate = {:.2f}'.format(p[0]))
loglog(Nv, exp(polyval(p,log(Nv))), '--')

# WPA w/lim
p = polyfit(log(Nv), log(e2_wpaw.astype(float)), 1)
loglog(Nv, e2_wpaw, '.-',markersize=10, label = 'WPA, rate = {:.2f}'.format(p[0]))
loglog(Nv, exp(polyval(p,log(Nv))), '--')
    
set_xticks(Nv)
xlabel('# Elements')
ylabel('Error (L2-error)')
title('Error vs number of DOF (exact, w/lim)')

grid(axis='both',linestyle='--')
legend()
show()   

In [ ]:
figure(6)
clf()   

Nv = df.index.levels[4].values

# DG exact 
p = polyfit(log(Nv), log(e2_dgi.astype(float)), 1)
loglog(Nv, e2_dgi, '.-',markersize=10, label = 'DG, rate = {:.2f}'.format(p[0]))
loglog(Nv, exp(polyval(p,log(Nv))), '--')

# CG exact
p = polyfit(log(Nv), log(e2_cgi.astype(float)), 1)
loglog(Nv, e2_cgi, '.-',markersize=10, label = 'CG, rate = {:.2f}'.format(p[0]))
loglog(Nv, exp(polyval(p,log(Nv))), '--')

# WPA w/lim
p = polyfit(log(Nv), log(e2_wpano.astype(float)), 1)
loglog(Nv, e2_wpano, '.-',markersize=10, label = 'WPA, rate = {:.2f}'.format(p[0]))
loglog(Nv, exp(polyval(p,log(Nv))), '--')
    
set_xticks(Nv)
xlabel('# Elements')
ylabel('Error (L2-error)')
title('Error vs number of DOF (inexact, wout/lim)')

grid(axis='both',linestyle='--')
legend()
show()   